# Index based strategy

In this strategy we will use the index data to build output matrix.

In [ ]:
# Import basic libraries for manipulating data.

# Please refer to xarray.pydata.org for xarray documentation.

# xarray works optimally with N-dimensional datasets in Python
# and is well suited for financial datasets with labels "time",
# "field" and "asset". xarray data structures can also be easily
# converted to pandas dataframes.

import xarray as xr

import numpy as np
import pandas as pd

# Import quantnet libraries.

import qnt.data as qndata          # data loading and manipulation
import qnt.stepper as qnstepper    # strategy definition
import qnt.stats as qnstats        # key statistics
import qnt.graph as qngraph        # graphical tools
import qnt.forward_looking as qnfl # forward looking checking
import qnt.ta as qnta

# display function for fancy displaying:
from IPython.display import display

import datetime as dt

In [ ]:
# Load all available asset names since given date.

assets = qndata.load_assets(min_date="2010-01-01")

assets_names = [i["id"] for i in assets]

# Load all available data since given date.

# It is possible to set a max_date in the call in order to
# develop the system on a limited in-sample period and later
# test the system on unseen data after max_date.

# A submission will be accepted only if no max_date is set,
# as submissions will be evaluated on live data on a daily basis.

data = qndata.load_data(tail=dt.timedelta(days=5*365),
                        dims=("time", "field", "asset"),
                        assets=assets_names,
                        forward_order=True)

Now load the index data. We will use RUSSEL-2000 (RUT).

In [ ]:
# Load available index list (just for demo)
index_list = qndata.load_index_list()
display("available indexes", index_list)

index_name = 'RUT'

index_data = qndata.load_index_data(ids=[index_name], tail=dt.timedelta(days=5*365), forward_order=True)
display("index data", index_data.to_pandas())

index_data = index_data.sel(asset=index_name)
# align index data, this is necessary for further calculations
index_data = xr.align(index_data, data.isel(field=0), join='right')[0].ffill('time')

We will use BETA coefficient as risk factor:

In [ ]:
def beta(prices, index, periods=252, dim='time'):
    result = prices.copy(True)
    for a in prices.asset.values:
        result.loc[{"asset":a}] = qnta.beta(prices.loc[{"asset":a}], index, periods)
    return result

Now, let's define the strategy:

In [ ]:
close = data.sel(field='close')

assets_beta = beta(close, index_data, 250)

roc = qnta.sroc(close, 50, 250)

output = data.loc[:,"is_liquid",:].where(roc > 0).where(assets_beta < 0.7)
output /= output.sum('asset')
output = xr.where(output > 0.05, 0.05, output)

# display stats tail
stat = qnstats.calc_stat(data, output)
display(stat.to_pandas().tail())

# Statistics

In [ ]:
# Calculate statistics on a rolling basis.

# Transactions are punished with slippage equal to a given
# fraction of ATR14 (read more about slippage in our full
# Strategy Buy and Hold template). We evaluate submissions
# using 5% of ATR14 for slippage.

# Mean return, volatility and Sharpe ratio are computed on a
# rolling basis using a lookback period of 3 years.

stat = qnstats.calc_stat(data, output)

display(stat.to_pandas().tail())

In [ ]:
def print_stat(stat):
    """Prints selected statistical key indicators:
       - the global Sharpe ratio of the strategy;
       - the global mean profit;
       - the global volatility;
       - the maximum drawdown.

       Note that Sharpe ratio, mean profit and volatility
       apply to  max simulation period, and not to the
       rolling basis of 3 years.
    """

    days = len(stat.coords["time"])
    
    returns = stat.loc[:, "relative_return"]

    equity = stat.loc[:, "equity"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = (qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    volatility = (qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    max_ddown = (qnstats.calc_max_drawdown(
        qnstats.calc_underwater(equity)).to_pandas().values[-1])*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility [%]       : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

print_stat(stat)

In [ ]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

In [ ]:
# show underwater chart:
UWchart = stat.to_pandas()["underwater"]
qngraph.make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart", range_max=0)

In [ ]:
# show rolling Sharpe ratio on a 3-year basis:
SRchart = stat.to_pandas()["sharpe_ratio"].iloc[(252*3):]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

In [ ]:
# show bias chart:
biaschart = stat.to_pandas()["bias"]
qngraph.make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

## Checks

In [ ]:
# correlation check
# your strategy should not correlate with other strategies before submission
qnstats.print_correlation(output, data)

# Write output

In [ ]:
# Finally, we write the last mandatory step for submission,
# namely writing output to file:

qndata.write_output(output)

At this stage code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!